<a href="https://colab.research.google.com/github/miguelbeca/PythonDataScienceHandbook/blob/master/Restaurante_Queue_Discrete_Event_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from queue import Queue
from random import seed
from random import randint

In [0]:
#Define class to hold customer events
class CustomerEvent:
    def __init__(self, customer_id, event, arrival_time, departure_time=None):
        self.customer_id = customer_id #customer id
        self.event = event # can be either arrival or departure
        self.arrival_time = arrival_time
        self.departure_time = departure_time
        
    def printCustomerEvent(self):
        print("Customer ID: {}".format(self.customer_id))
        print("Event: {}".format(self.event))
        print("Arrival Time: {}".format(self.arrival_time))
        print("Departure Time: {}".format(self.departure_time))

In [0]:
#Define class Restaurant to hold global variables
class Restaurant:
    def __init__(self):
        self.server_busy = False
        self.FEL = []
        self.wait_queue = Queue()
        self.clock = 0
        self.customer_id = 1 #used to generate unique customer ID numbers
        self.customer_count = 1
        self.max_customers = 10
        self.customer_stats = []

In [0]:
#Function to print out contents of Future Events List
#Used for debugging
def print_FEL(FEL):
    for event in FEL:
        event.printCustomerEvent()

In [0]:
#Variables Initialization
#Initialize global variables
rest = Restaurant()
seed(10) #Random number generator seed

In [0]:
#Model Arrivals function
def model_arrival(cust_event, rest):

    if (rest.customer_count < rest.max_customers):
        #Add next arrival event to EFL
        arrival_time2 = rest.clock + randint(1,60)
        new_customer = CustomerEvent(rest.customer_id, 'arrival', arrival_time2)
        rest.FEL.append(new_customer)
        rest.customer_id +=1
        rest.customer_count +=1
        #If server is busy add it to the wait queue
        if (rest.server_busy == True):
            rest.wait_queue.put(cust_event)
        else:
            rest.server_busy = True
            #schedule customer departure and add to FEL
            cust_event.event = 'departure'
            departure_time = cust_event.arrival_time + randint(1,60)
            cust_event.departure_time = departure_time
            rest.FEL.append(cust_event)

In [0]:
#Model departures function
def model_departure(cust_event, rest):
    #Get stats on customer
    cust_statistics = {'id':cust_event.customer_id,
                       'arrival': cust_event.arrival_time,
                       'departure': cust_event.departure_time,
                       'time_spent': (cust_event.departure_time-cust_event.arrival_time)
                        }
    #Add individual statistics to list
    rest.customer_stats.append(cust_statistics)
    
    #If queue is empty, then server is set to idle. Otherwise, process events in queue
    if (rest.wait_queue.empty()):
        rest.server_busy = False
    else:
        wait_list_event = rest.wait_queue.get()
        #schedule customer departure and add to FEL
        wait_list_event.event = 'departure'
        departure_time = wait_list_event.arrival_time + randint(1,60)
        wait_list_event.departure_time = departure_time
        rest.FEL.append(wait_list_event)

In [0]:
#Generate first customer arrival
arrival_time_1 = rest.clock + randint(1,60)
customer_1 = CustomerEvent(rest.customer_id, 'arrival', arrival_time_1)
rest.customer_id +=1

#Add first arrival to FEL
rest.FEL.append(customer_1)

In [9]:
#Main loop
while rest.FEL:
    cust_event = rest.FEL[0] #since we append events to the list, first event is the one to be processed
    rest.FEL.pop(0)
    rest.clock = cust_event.arrival_time
    if (cust_event.event == 'arrival'):
        model_arrival(cust_event, rest)
    else:
        model_departure(cust_event, rest)

#Print customer statistics
time_spent_sum = 0
for dict_item in rest.customer_stats:
        print("Customer ID: {} - Arrival Time: {} - Departure Time: {} - Time spent: {}".format(dict_item['id'], \
               dict_item['arrival'],  dict_item['departure'], dict_item['time_spent']))
        time_spent_sum += dict_item['time_spent']
nr_customers_served = len(rest.customer_stats)
print("Total number of customers: {}".format(nr_customers_served))
print("Average time spent per customer: {:.2f}".format(time_spent_sum/nr_customers_served))

Customer ID: 1 - Arrival Time: 37 - Departure Time: 65 - Time spent: 28
Customer ID: 2 - Arrival Time: 40 - Departure Time: 77 - Time spent: 37
Customer ID: 3 - Arrival Time: 71 - Departure Time: 85 - Time spent: 14
Customer ID: 4 - Arrival Time: 72 - Departure Time: 125 - Time spent: 53
Customer ID: 5 - Arrival Time: 102 - Departure Time: 155 - Time spent: 53
Customer ID: 6 - Arrival Time: 134 - Departure Time: 176 - Time spent: 42
Customer ID: 7 - Arrival Time: 152 - Departure Time: 163 - Time spent: 11
Customer ID: 8 - Arrival Time: 204 - Departure Time: 238 - Time spent: 34
Customer ID: 9 - Arrival Time: 207 - Departure Time: 228 - Time spent: 21
Total number of customers: 9
Average time spent per customer: 32.56
